## 对user_info.txt的EDA和预处理
除了userId，其他列均有缺失值。

对于OS、province、city：添加一个"Unknown"类别。

对于age、gender：概率均分。

共有1538384个用户。

输出user_info.pkl。

In [1]:
import gc
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from time import time
pd.set_option('expand_frame_repr', False)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 1000)
pd.set_option('max_colwidth', 200)

In [2]:
user_path = '../data/user_info.txt'
st = time()
user_df = pd.read_csv(user_path, header=None, sep='\t',
                    names=['userId', 'deviceName', 'OS', 'province', 'city', 'age', 'gender'],
                    dtype={'userId': 'category', 'deviceName': 'category', 'OS': 'category', 'province': 'category',
                           'city': 'category', 'age': 'object', 'gender': 'object'})
ed = time()
print(user_df.info())
print(f'read csv: {ed - st: .2f}s')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1538384 entries, 0 to 1538383
Data columns (total 7 columns):
 #   Column      Non-Null Count    Dtype   
---  ------      --------------    -----   
 0   userId      1538384 non-null  category
 1   deviceName  1470640 non-null  category
 2   OS          1470576 non-null  category
 3   province    1446124 non-null  category
 4   city        1442312 non-null  category
 5   age         1478493 non-null  object  
 6   gender      1479812 non-null  object  
dtypes: category(5), object(2)
memory usage: 83.7+ MB
None
read csv:  8.17s


In [3]:
# st = time()
# user_df = pd.read_pickle('user_info.pkl')
# ed = time()
# print(user_df.info())
# print(f'pickle: {ed - st: .2f}')
# user_df.head()

In [4]:
print(user_df.info())
print('-----------------------------------')
print(user_df.head())
print('-------------- --------------------')
print('NULL values count in each column:')
print(user_df.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1538384 entries, 0 to 1538383
Data columns (total 7 columns):
 #   Column      Non-Null Count    Dtype   
---  ------      --------------    -----   
 0   userId      1538384 non-null  category
 1   deviceName  1470640 non-null  category
 2   OS          1470576 non-null  category
 3   province    1446124 non-null  category
 4   city        1442312 non-null  category
 5   age         1478493 non-null  object  
 6   gender      1479812 non-null  object  
dtypes: category(5), object(2)
memory usage: 83.7+ MB
None
-----------------------------------
       userId deviceName       OS province city                                                               age                         gender
0  1000372820   TAS-AN00  Android       广东   广州  A_0_24:0.404616,A_25_29:0.059027,A_30_39:0.516550,A_40+:0.019806  female:0.051339,male:0.948661
1  1000652892     PACM00  Android       河北   唐山  A_0_24:0.615458,A_25_29:0.086233,A_30_39:0.141408,A_40+:0.

In [5]:
userId = user_df['userId']
print(f'total {userId.size} users.')
userId.value_counts(dropna=False, )

total 1538384 users.


1000372820    1
2400553116    1
2400558968    1
2400558508    1
2400557464    1
             ..
2445826628    1
2445826006    1
2445825836    1
2445825782    1
999902420     1
Name: userId, Length: 1538384, dtype: int64

In [6]:
os = user_df['OS']
os = os.cat.add_categories('Unknown')
os.fillna('Unknown', inplace=True)
user_df.OS = os
print(os.value_counts(dropna=False))

Android    1455464
Unknown      67808
IOS          15112
Name: OS, dtype: int64


In [7]:
print(os.cat.categories)

Index(['Android', 'IOS', 'Unknown'], dtype='object')


In [8]:
province = user_df['province']
province = province.cat.add_categories('Unknown')
province.fillna('Unknown', inplace=True)
user_df.province = province
print(province.value_counts(dropna=False))

广东                        183972
江苏                         96784
山东                         94111
Unknown                    92260
河北                         82733
浙江                         77201
四川                         69178
河南                         68337
湖南                         60286
湖北                         53694
广西                         52632
安徽                         51437
辽宁                         48626
上海                         45722
福建                         44998
云南                         43984
北京                         40937
江西                         37767
山西                         36829
陕西                         32118
重庆                         28527
贵州                         28157
黑龙江                        26256
内蒙古                        23769
天津                         23150
新疆                         21336
吉林                         21220
甘肃                         20489
海南                         15230
宁夏                          6231
青海        

In [9]:
city = user_df['city']
city = city.cat.add_categories('Unknown')
city.fillna('Unknown', inplace=True)
user_df.city = city
print(city.value_counts(dropna=False))

Unknown                 96072
上海                      45722
北京                      40937
广州                      31825
重庆                      28527
成都                      24619
天津                      23150
东莞                      22484
深圳                      21942
武汉                      19327
苏州                      18765
佛山                      17909
杭州                      17636
西安                      14886
石家庄                     14154
南京                      13028
济南                      12732
保定                      11994
大连                      11968
郑州                      11820
长沙                      11780
宁波                      11764
青岛                      11376
昆明                      11328
南宁                      11254
沈阳                      11053
合肥                      10171
哈尔滨                      9958
唐山                       9909
无锡                       9820
福州                       9592
惠州                       9296
泉州                       9160
长春        

In [10]:
# 某些行列表长度为16，把每一个元素额外复制了3遍。
# 此处处理相当于[0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3] -> [0, 1, 2, 3]
age = user_df['age']
notnull = age[age.notnull()]
redundant_rows = notnull[notnull.str.split(',').map(len) == 16]
print(redundant_rows.iloc[:3].values)
redundant_rows = redundant_rows.str.split(',').map(lambda x: ','.join(x[::4]))
age[redundant_rows.index] = redundant_rows  # setting with copy warning
print('--->')
print(age[redundant_rows.index].iloc[:3].values)

['A_0_24:0.256968,A_0_24:0.256968,A_0_24:0.256968,A_0_24:0.256968,A_25_29:0.129631,A_25_29:0.129631,A_25_29:0.129631,A_25_29:0.129631,A_30_39:0.270219,A_30_39:0.270219,A_30_39:0.270219,A_30_39:0.270219,A_40+:0.343182,A_40+:0.343182,A_40+:0.343182,A_40+:0.343182'
 'A_0_24:0.129381,A_0_24:0.129381,A_0_24:0.129381,A_0_24:0.129381,A_25_29:0.234456,A_25_29:0.234456,A_25_29:0.234456,A_25_29:0.234456,A_30_39:0.314669,A_30_39:0.314669,A_30_39:0.314669,A_30_39:0.314669,A_40+:0.321493,A_40+:0.321493,A_40+:0.321493,A_40+:0.321493'
 'A_0_24:0.131260,A_0_24:0.131260,A_0_24:0.131260,A_0_24:0.131260,A_25_29:0.290373,A_25_29:0.290373,A_25_29:0.290373,A_25_29:0.290373,A_30_39:0.357634,A_30_39:0.357634,A_30_39:0.357634,A_30_39:0.357634,A_40+:0.220733,A_40+:0.220733,A_40+:0.220733,A_40+:0.220733']
--->
['A_0_24:0.256968,A_25_29:0.129631,A_30_39:0.270219,A_40+:0.343182'
 'A_0_24:0.129381,A_25_29:0.234456,A_30_39:0.314669,A_40+:0.321493'
 'A_0_24:0.131260,A_25_29:0.290373,A_30_39:0.357634,A_40+:0.220733']


/environment/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [11]:
# 对于缺失值，将各个年龄段的概率设为0.25
null = age[age.isnull()]
print(null.iloc[:3].values)
age[null.index] = 'A_0_24:0.25000,A_25_29:0.25000,A_30_39:0.25000,A_40+:0.25000'
print('--->')
print(age[null.index].iloc[:3].values)

[nan nan nan]
--->
['A_0_24:0.25000,A_25_29:0.25000,A_30_39:0.25000,A_40+:0.25000'
 'A_0_24:0.25000,A_25_29:0.25000,A_30_39:0.25000,A_40+:0.25000'
 'A_0_24:0.25000,A_25_29:0.25000,A_30_39:0.25000,A_40+:0.25000']


/environment/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [12]:
# 所有行格式一致后，进行字符串的分割
age_df = age.str.split(',', expand=True).applymap(lambda x: float(x.split(':')[1]))
age_df.columns = ['0-24', '25-29', '30-39', '40-']
print('no null in age df!')
print(age_df.isnull().sum())

no null in age df!
0-24     0
25-29    0
30-39    0
40-      0
dtype: int64


In [13]:
# 同样地，性别也有被重复的行
# 相当于[0, 0, 1, 1] -> [0, 1]
gender = user_df['gender']
notnull = gender[gender.notnull()]
redundant_row = notnull[notnull.str.split(',').map(len) == 4]
print(redundant_row.iloc[:3].values)
redundant_row = redundant_row.str.split(',').map(lambda x: ','.join(x[::2]))
gender.iloc[redundant_row.index] = redundant_row  # setting with copy warning
print('--->')
print(gender[redundant_row.index].iloc[:3].values)

['female:0.852616,female:0.852616,male:0.147384,male:0.147384'
 'female:0.534680,female:0.534680,male:0.465320,male:0.465320'
 'female:0.559464,female:0.559464,male:0.440536,male:0.440536']
--->
['female:0.852616,male:0.147384' 'female:0.534680,male:0.465320'
 'female:0.559464,male:0.440536']


/environment/miniconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [14]:
# 某些样本在male/female=1.0时，不会写另一个的概率为0.0，这里补充上，保持格式一致

notnull = gender[gender.notnull()]
length = notnull.str.split(',').map(len)
length_1 = length[length == 1]
length_1 = gender[length_1.index]
print(length_1.iloc[:3].values)
extend = length_1.map(lambda x: "female:0.00000," + x if x.startswith('male') else x + ',male:0.00000')
gender[extend.index] = extend  # setting with copy warning
print('--->')
print(gender[extend.index].iloc[:3].values)

['male:1.000000' 'male:1.000000' 'male:1.000000']
--->
['female:0.00000,male:1.000000' 'female:0.00000,male:1.000000'
 'female:0.00000,male:1.000000']


/environment/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [15]:
# 对于缺失值，置female = male = 0.5
null = gender[gender.isnull()]
print(null.iloc[:3].values)
gender[null.index] = 'female:0.50000,male:0.50000'
print('--->')
print(gender[null.index].iloc[:3].values)

[nan nan nan]
--->
['female:0.50000,male:0.50000' 'female:0.50000,male:0.50000'
 'female:0.50000,male:0.50000']


/environment/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [16]:
gender_df = gender.str.split(',', expand=True).applymap(lambda x: float(x.split(':')[1]))
gender_df.columns = ['female', 'male']
print('no null values in gender df!')
print(gender_df.isnull().sum())

no null values in gender df!
female    0
male      0
dtype: int64


In [17]:
device = user_df.deviceName
print(device.nunique(), 'unique values')
print(device.value_counts(dropna=False))
device = device.cat.add_categories('Unknown')
device.fillna('Unknown', inplace=True)
user_df.deviceName = device

3095 unique values
PBAM00         149501
PBBM00          74098
PBEM00          73779
NaN             67744
PACM00          54418
                ...  
LG-H815             1
LG-M210             1
LM-V600             1
LenovoA5860         1
BLN-L24             1
Name: deviceName, Length: 3096, dtype: int64


/environment/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  


In [29]:
user_df = pd.concat([user_df.drop(['age', 'gender'], axis=1), age_df, gender_df], axis=1)
user_df.head()

,userId,deviceName,OS,province,city,0-24,25-29,30-39,40-,female,male
0,1000372820,TAS-AN00,Android,广东,广州,0.40,0.06,0.52,0.02,0.05,0.95
1,1000652892,PACM00,Android,河北,唐山,0.62,0.09,0.14,0.16,0.28,0.72
2,1000908852,MI6X,Android,上海,上海,0.12,0.21,0.30,0.37,0.00,1.00
3,1001168798,iPhone11,IOS,Unknown,Unknown,0.44,0.49,0.06,0.01,0.87,0.13
4,1001305614,M2103K19C,Android,江苏,苏州,0.01,0.04,0.35,0.60,0.00,1.00


In [30]:
print(user_df.isnull().sum())


userId        0
deviceName    0
OS            0
province      0
city          0
0-24          0
25-29         0
30-39         0
40-           0
female        0
male          0
dtype: int64


In [35]:
user_df.to_pickle('~/data/user.pkl')